In [1]:
import pandas as pd
import numpy as np

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier 

In [2]:
train_set = pd.read_pickle('train_set.pkl')
strat_train_set = pd.read_pickle('strat_train_set.pkl')
strat_test_set = pd.read_pickle('strat_test_set.pkl')
strat_valid_set = pd.read_pickle('strat_valid_set.pkl')

Testing `MultinomialNB` and `KNeighborsClassifier` with best parameters selected from gridsearch, 
on both validation set `strat_valid_set` and test set `strat_test_set` (check for over-fitting)

In [18]:
mnb_tfidf = TfidfVectorizer(lowercase=False, min_df=10, max_df=0.5, use_idf=True,
                        analyzer='char', ngram_range=(2,4))
mnb = MultinomialNB()

In [19]:
mnb_pipe = Pipeline(
    steps=[('tfidf', mnb_tfidf), 
           ('mnb', mnb)]
)

In [20]:
mnb_pipe.fit(strat_train_set['cleaned'].values, strat_train_set['tags_id'])
#Validation set
mnb_pipe.score(strat_valid_set['cleaned'].values, strat_valid_set['tags_id'])

0.9871630295250321

In [26]:
print(classification_report(strat_valid_set['tags_id'], 
                            mnb_pipe.predict(strat_valid_set['cleaned'].values), 
                            target_names=['梁振英', '美國大選', '足球']))

              precision    recall  f1-score   support

         梁振英       0.99      0.96      0.97       186
        美國大選       0.97      0.99      0.98       169
          足球       0.99      1.00      1.00       424

   micro avg       0.99      0.99      0.99       779
   macro avg       0.98      0.98      0.98       779
weighted avg       0.99      0.99      0.99       779



In [21]:
#Test set
mnb_pipe.score(strat_test_set['cleaned'].values, strat_test_set['tags_id'])

0.9922978177150192

In [27]:
print(classification_report(strat_test_set['tags_id'], 
                            mnb_pipe.predict(strat_test_set['cleaned'].values), 
                            target_names=['梁振英', '美國大選', '足球']))

              precision    recall  f1-score   support

         梁振英       0.98      0.99      0.99       186
        美國大選       0.99      0.99      0.99       168
          足球       1.00      0.99      0.99       425

   micro avg       0.99      0.99      0.99       779
   macro avg       0.99      0.99      0.99       779
weighted avg       0.99      0.99      0.99       779



In [14]:
neigh_tfidf = TfidfVectorizer(lowercase=False, min_df=20, max_df=0.75, max_features=2000,
                              use_idf=False, analyzer='char', ngram_range=(1,4))
neigh = KNeighborsClassifier(n_neighbors=3)

In [15]:
neigh_pipe = Pipeline(
    steps=[('tfidf', neigh_tfidf), 
           ('neigh', neigh)]
)

In [22]:
neigh_pipe.fit(strat_train_set['cleaned'].values, strat_train_set['tags_id'])
#Validation set
neigh_pipe.score(strat_valid_set['cleaned'].values, strat_valid_set['tags_id'])

0.9807445442875481

In [28]:
print(classification_report(strat_valid_set['tags_id'], 
                            neigh_pipe.predict(strat_valid_set['cleaned'].values), 
                            target_names=['梁振英', '美國大選', '足球']))

              precision    recall  f1-score   support

         梁振英       0.99      0.96      0.97       186
        美國大選       0.94      0.99      0.97       169
          足球       1.00      0.99      0.99       424

   micro avg       0.98      0.98      0.98       779
   macro avg       0.97      0.98      0.98       779
weighted avg       0.98      0.98      0.98       779



In [23]:
#Test set
neigh_pipe.score(strat_test_set['cleaned'].values, strat_test_set['tags_id'])

0.9833119383825417

In [29]:
print(classification_report(strat_test_set['tags_id'], 
                            neigh_pipe.predict(strat_test_set['cleaned'].values), 
                            target_names=['梁振英', '美國大選', '足球']))

              precision    recall  f1-score   support

         梁振英       1.00      0.98      0.99       186
        美國大選       0.93      1.00      0.96       168
          足球       1.00      0.98      0.99       425

   micro avg       0.98      0.98      0.98       779
   macro avg       0.98      0.99      0.98       779
weighted avg       0.98      0.98      0.98       779



Both `MultinomialNB` and `KNeighborsClassifier` do not appear to be over-fitting. 

Since `MultinomialNB` has a higher accuracy, we will the classification with `MultinomialNB`

# Evaluation on the unlablled data set

In [112]:
labels = { 0: '梁振英', 1: '美國大選', 2: '足球' }

In [41]:
from ipynb.fs.full.DataPreprocessing import clean_text

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.747 seconds.
Prefix dict has been built succesfully.


In [47]:
test_set = pd.read_csv('offsite-test-material/offsite-tagging-test-set (1).csv')

In [49]:
test_set['cleaned'] = test_set['text'].apply(lambda x: clean_text(x) )

In [76]:
mnb_tfidf = TfidfVectorizer(lowercase=False, min_df=10, max_df=0.5, use_idf=True,
                        analyzer='char', ngram_range=(2,4))
mnb = MultinomialNB()

mnb_pipe = Pipeline(
    steps=[('tfidf', mnb_tfidf), 
           ('mnb', mnb)]
)
mnb_pipe.fit(strat_train_set['cleaned'].values, strat_train_set['tags_id'].values)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=0.5, max_features=None, min_df=10,
        ngram_range=(2, 4), norm='l2', preprocessor=None, smooth_idf=Tru...rue,
        vocabulary=None)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [77]:
vocbas = mnb_pipe.named_steps['tfidf'].vocabulary_
vocbas = {v: k for k, v in vocbas.items()}

In [124]:
test_set['predicted_prob'] = list(mnb_pipe.predict_proba(test_set['cleaned'].values))
test_set['predicted_tag'] = list(mnb_pipe.predict(test_set['cleaned'].values))
test_set['confidence'] = test_set.apply(lambda row: row['predicted_probs'][row['predicted_tag']], axis=1)

In [133]:
test_set.head()

,id,text,cleaned,predicted_prob,predicted,predicted_probs,predicted_tag,confidence,predicted_tag_id
0,6,南華添鋒力 簽前厄瓜多爾國腳保耶 港超勁旅南華宣布羅致前厄瓜多爾國腳菲力斯保耶（Felix ...,南華添鋒力簽前厄瓜多爾國腳保耶 港超勁旅南華宣布羅致前厄瓜多爾國腳菲力斯保耶（Felix B...,"[0.0011544558688102852, 0.0010776610054577336,...",2,"[0.0011544558688102852, 0.0010776610054577336,...",足球,0.997768,2
1,128,如果大學$0捐款 科大嶺南將蝕過千萬元 據now新聞台報道，身兼8大校監的特首梁振英曾以大學...,如果大學$捐款科大嶺南將蝕過千萬元 據now新聞台報道，身兼大校監的特首梁振英曾以大學已有足...,"[0.9538067077465673, 0.03062032029780316, 0.01...",0,"[0.9538067077465673, 0.03062032029780316, 0.01...",梁振英,0.953807,0
2,136,英超最強火力對碰 雙城爭冠靠鋒霸 英超今季風起雲湧，傳統「Big 5」只剩兩隊名列積分榜前5...,英超最強火力對碰雙城爭冠靠鋒霸 英超今季風起雲湧，傳統「Big 」只剩兩隊名列積分榜前。明晨...,"[3.890262424990809e-06, 4.174607502688508e-06,...",2,"[3.890262424990809e-06, 4.174607502688508e-06,...",足球,0.999992,2
3,213,【01球評】膺半程冠軍 阿仙奴今季不奪標更待何時？ 近年「兵工廠」每季的起落都離不開一個循環...,球評膺半程冠軍阿仙奴今季不奪標更待何時？ 近年「兵工廠」每季的起落都離不開一個循環：出閘脫腳...,"[4.182208520285169e-06, 2.858175206766063e-06,...",2,"[4.182208520285169e-06, 2.858175206766063e-06,...",足球,0.999993,2
4,658,【書商失蹤】梁振英：希望失蹤的李波本人提供資料 行政長官梁振英出席行政會議前見記者，被問及李...,書商失蹤梁振英：希望失蹤的李波本人提供資料 行政長官梁振英出席行政會議前見記者，被問及李波傳...,"[0.9988663842325067, 0.000822657926727919, 0.0...",0,"[0.9988663842325067, 0.000822657926727919, 0.0...",梁振英,0.998866,0


### Eye-balling the predictions with lowest confidence

In [127]:
test_set['predicted_tag'] = test_set['predicted_tag'].apply(lambda x: labels[x])

In [134]:
#For label 0 '梁振英'
test_set[test_set['predicted_tag']=='梁振英'].sort_values(by='confidence').head(20)\
[['cleaned', 'predicted_tag', 'confidence']]

,cleaned,predicted_tag,confidence
711,特首選戰上傳自殘照稱遭家暴梁振英曾認齊昕健康有問題 疑?手自殘齊昕於年月下旬，突在Faceb...,梁振英,0.431293
132,CY 叫白領跳落維港游水 據說中環白領 lunch time 食完飯，仲會「剩下 分鐘」，...,梁振英,0.464966
695,英媒揭足總曾中止保護兒童免遭性侵計劃做法「非常令人關注」 英格蘭足總是在年推行一項為期年的研...,梁振英,0.509131
481,蔡東豪專欄如果梁振英是CEO不是特首，曾俊華捱到幾耐？ 美國總統候選人特朗普認為自己是最合適...,梁振英,0.527062
956,梁振英送蘭花證青協總幹事王鳴退休 青協職員稱不捨 記者於活動後追問王有關退休原因，她未有否...,梁振英,0.585624
474,海洋垃圾佔領石澳泳灘驚現兩個製冷劑氣罐恐有爆炸風險 近日主要吹東北風，本周日石澳也因大浪掛起...,梁振英,0.622948
154,足球政情梁振英fb「貼波」讚港足雖敗猶榮惹近萬「嬲嬲」 香港隊在一片歡呼聲中光榮出局，雖然暫...,梁振英,0.647152
509,劉鳴煒捐億卡羅琳醫學院復修醫學中心成立聘人設科研團隊 卡羅琳醫學院劉鳴煒復修醫學中心將由曾研...,梁振英,0.650403
740,梁振英賀傅家俊逆轉勝：完美體現港人永不放棄逆境自強精神 香港桌球好手傅家俊在蘇格蘭桌球公開賽...,梁振英,0.653382
214,蝸居港人房住不起窮人淪網吧難民 「為何在今時今日的香港，想要找到一個可以稱之為『家』的地方，...,梁振英,0.703657


In [135]:
#For label 2 '美國大選'
test_set[test_set['predicted_tag']=='美國大選'].sort_values(by='confidence').head(20)\
[['cleaned', 'predicted_tag', 'confidence']]

,cleaned,predicted_tag,confidence
643,特朗普當選極速消化狂人效應歐美股市倒升道指升點 道指低開點後回穩，並持續向好，最高曾升點，最...,美國大選,0.423515
438,加拿大總理杜魯多訪港先見李嘉誠後見特首 杜魯多今早與「誠哥」會面，多亦相當「識做」，在twi...,美國大選,0.600032
639,特朗普當選‧影片李安頹住講白宮新主人：地球像要毀滅了 台灣名導李安今日來港宣傳新片《比利．林...,美國大選,0.807929
503,美國大選紅星發力撐選舉里安納度愛瑪史東籲選民上網登記 還有一個月左右就是美國大選的日子，有鑑...,美國大選,0.847713
678,有其爺必有其孫？特朗普祖父逃兵役遭德國驅逐求情信曝光 德國《圖片報》報道，歷史學家找到特朗普...,美國大選,0.849686
558,華艦首泊金蘭灣折射美國重返亞洲陷死胡同美艦揀西沙巡航有因 載有名官兵的中國海軍編隊湘潭艦、舟...,美國大選,0.861943
621,從fashion 看大選米歇爾奧巴馬的fashion sense 米歇爾被譽為最有fashi...,美國大選,0.864443
557,美知識型經濟城市崛起 人口增多收入不升反跌 年月，歲的佩里（Alfred Perry）從工業...,美國大選,0.867472
572,北約東歐擴軍制衡俄國戰後最大規模 北約秘書長：有節制回應俄國擴張北約周三在布魯塞爾舉行防長會...,美國大選,0.893944
635,有片CNN黑人評論員心碎了：這是白人對黑人總統的反彈 雖然直播之際，大選結果還未有定案，但大...,美國大選,0.918745


In [136]:
#For label 2 '足球'
test_set[test_set['predicted_tag']=='足球'].sort_values(by='confidence').head(20)\
[['cleaned', 'predicted_tag', 'confidence']]

,cleaned,predicted_tag,confidence
164,熱話北角球場貼「勿進行球類活動」告示網民嘲：國際笑話 康樂及文化事務署球場不能打球？北角配水...,足球,0.638854
244,英超．圖輯烏普頓公園的最後一晚 Upton Park奏出Beatles名曲Twist and...,足球,0.655933
269,早晨精讀忌廉哥尖東最後一日大肚護士有種起義 忌廉哥告別粉絲不捨明星貓「忌廉哥」長期駐守的尖東...,足球,0.667923
493,傑志會長伍健回應收地起樓「香港足球未來必受重大挫折」 去年月正式啟用的賽馬會傑志中心，近日因...,足球,0.671447
684,球壇性侵風波擴大英警介入調查克魯前管理層爆球會早知情 繼活禾特後，另外兩名前克魯球員基斯安斯...,足球,0.736925
379,即日履新 Nike Magista 人類腳掌的演化 如果你相信進化論的話，都會明白所有生...,足球,0.748432
167,向種族歧視說不德國地區隊「染黑」撐隊友 球壇的歧視問題從沒解決，種族與性別歧視時有發生，但我...,足球,0.802494
752,哥倫比亞墜機哥倫比亞公布初步調查直斥「全因人為失誤」 上月在哥倫比亞發生的空難事故，造成巴西...,足球,0.891430
688,中村狂熱份子（下）：夢想之行是怎樣煉成的？ 聽到 Kengo 的故事，不免會問， 年時他以遊...,足球,0.898155
162,世盃外圍賽大公報：國足釀換帥足協：報道失實影響備戰 中國足協今日發聲明，指香港《大公報》早前...,足球,0.906160


In [131]:
test_set[['text', 'predicted_tag']].to_csv('offsite-tagging-test-set-tagged (1).csv')

In [132]:
test_set[['text', 'predicted_tag']]

,text,predicted_tag
0,南華添鋒力 簽前厄瓜多爾國腳保耶 港超勁旅南華宣布羅致前厄瓜多爾國腳菲力斯保耶（Felix ...,足球
1,如果大學$0捐款 科大嶺南將蝕過千萬元 據now新聞台報道，身兼8大校監的特首梁振英曾以大學...,梁振英
2,英超最強火力對碰 雙城爭冠靠鋒霸 英超今季風起雲湧，傳統「Big 5」只剩兩隊名列積分榜前5...,足球
3,【01球評】膺半程冠軍 阿仙奴今季不奪標更待何時？ 近年「兵工廠」每季的起落都離不開一個循環...,足球
4,【書商失蹤】梁振英：希望失蹤的李波本人提供資料 行政長官梁振英出席行政會議前見記者，被問及李...,梁振英
5,【施政盤點】三份施政報告 僅一半政策達標 行政長官梁振英即將公布任內第四份施政報告，《香港0...,梁振英
6,【施政盤點】「治港絕招」 設19委員會 空談多實務少 行政長官梁振英上任3年多，先後成立多個...,梁振英
7,高普首簽 「新馬迪」來季投紅軍 利物浦傷兵滿營及戰績不穩，主帥高普仍不忘投資未來，昨以510...,足球
8,「最潮主帥」鬥利物浦：我已領先1：0 英乙球隊埃克塞特在明晨的足總盃於主場迎戰利物浦，雖然越...,足球
9,紅軍超殘陣逼和英乙隊 高普：負擔不起重賽 逾十名球員受傷的利物浦，今晨在足總盃第三圈以大部份...,足球
